In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC as SupportVectorClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from TextFormatter import Formatter
import numpy as np
from itertools import count
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
import nltk
import re
import contractions
import pandas as pd
import numpy as np

In [3]:
training = {}
testing = {}
empires = []

with open("Resources/Data/Training.txt", "r", encoding="utf8") as file:
    empires = file.readline().split(" ")
    for empire in empires:
        training[empire] = []
        testing[empire] = []
        empires.append(empire)

In [ ]:
def make_dataset(self, file, data_dict):
    current_empire = None
    for line in file.readlines()[1:]:
        line = self.clean_whitespace(line)
        line = self.fix_era(line)

        if line in self.empires:
            current_empire = line
            continue
        elif len(line) < 7 or "source" in "".join(line).lower():
            continue
        elif line[-1] != ".":
            line += "."

        line = nltk.sent_tokenize(line)

        for sentence in line:
            sentence = self.format_sentence(sentence, current_empire)
            if sentence is not None:
                data_dict[current_empire].extend([sentence])
    return data_dict

In [ ]:
with open("Resources/Data/Training.txt", "r", encoding="utf8") as file:
    training = make_dataset(file, training)

with open("Resources/Data/testing.txt", "r", encoding="utf8") as file:
    testing = make_dataset(file, testing)

In [ ]:
vectorizer = CountVectorizer()
text = list()
for empire in empires:
    text.extend(training[empire])
    text.extend(testing[empire])
vectorizer.fit(text)

In [ ]:
def get_pos(tag):
    if tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif tag.startswith('R'):
         return nltk.corpus.wordnet.ADV
    else:
        return nltk.corpus.wordnet.NOUN

In [ ]:
def preprocess_data(self, data):
    text = []
    labels = []
    for empire in self.empires:
        for sentence in data[empire]:
            wnl = WordNetLemmatizer()
            sentence = nltk.word_tokenize(sentence)
            sentence = nltk.pos_tag(sentence)
            sentence = [wnl.lemmatize(word, self.get_wordnet_pos(pos)) for word, pos in sentence]
            text.append(sentence)
            label = np.zeros(len(self.empires))
            label[self.empires.index(empire)] = 1
            labels.append(label)
    return text, labels

In [ ]:
formatter = Formatter()

training_data_text, training_labels = preprocess_data(training)
training_data = formatter.vectorize(training_data_text)

testing_data_text, testing_labels = preprocess_data(testing)
testing_data = formatter.vectorize(testing_data_text)

In [ ]:
sentences = []
labels = []
for empire in empires:
    sentences.append(training[empire][0:2])
    labels.append(empire)
data = pd.DataFrame({"Text":sentences, "Labels":labels})
data.head()

NameError: name 'empires' is not defined

In [ ]:
x_axis = []
y_axis = []
for depth in range(100):
    decision_tree = DecisionTreeClassifier(random_state=2, max_depth=depth)
    decision_tree.fit(training_data, training_labels)
    score = decision_tree.score(testing_data, testing_labels)
    x_axis.append(depth)
    y_axis.append(score)
plt.Line2D(x_axis, y_axis)
plt.show()

In [ ]:
x_axis = []
y_axis = []
for estimators in range(100):
    for depth in range(100):
        random_forest = RandomForestClassifier(n_estimators=estimators, random_state=14, max_depth=depth)
        random_forest.fit(training_data, training_labels)
        score = random_forest.score(testing_data, testing_labels)
        x_axis.append(depth)
        y_axis.append(score)